In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
data = pd.ExcelFile('dataset/ad_ranking_clean.xlsx')
ads = pd.read_excel(data, sheet_name='Ads', header=0)
mods = pd.read_excel(data, sheet_name='Mods', header=0)

In [29]:
# Decision weights used
latest_weight = 20
start_weight = 20
adrev_weight = 35
st_weight = 25

In [30]:
# Ad Revenue Score
ads["adrev_diff"] = ads["ad_revenue"] - ads["avg_ad_revenue"]
adrev_max = ads["adrev_diff"].max()
adrev_min = ads["adrev_diff"].min()
ads["adrev_score"] = (ads["adrev_diff"] - adrev_min) / (adrev_max - adrev_min) * adrev_weight

In [31]:
# Baseline ST Score
st_max = ads["baseline_st"].max()
st_min = ads["baseline_st"].min()
ads["st_score"] = (st_max - ads["baseline_st"]) / (st_max - st_min) * st_weight

In [32]:
# Latest Punish Score
ads['p_date_dateform'] = pd.to_datetime(ads['p_date'], format='%Y%m%d')
ads['days_from_latest_to_p'] = (ads['p_date_dateform'] - ads['latest_punish_begin_date']).dt.days
latest_max = ads["days_from_latest_to_p"].max()
latest_min = ads["days_from_latest_to_p"].min()
# ads['latest_punish_score'] = ((ads["days_from_latest_to_p"] - latest_min) / (latest_max - latest_min) * latest_weight) / ads["punish_num"]
ads['latest_punish_score'] = ((ads["days_from_latest_to_p"] - latest_min) / (latest_max - latest_min) * latest_weight) / np.where(ads["punish_num"] > 0, ads["punish_num"], 1)

In [33]:
# Start Time Score
ads['days_from_start_to_p'] = (ads['p_date_dateform'] - ads['start_time']).dt.days
start_max = ads["days_from_start_to_p"].max()
start_min = ads["days_from_start_to_p"].min()
ads["start_score"] = (start_max - ads["days_from_start_to_p"]) / (start_max - start_min) * start_weight

In [34]:
# Total Score
ads["total_score"] = ads["start_score"] + ads["latest_punish_score"] + ads["st_score"] + ads["adrev_score"]

# Apply Min-Max scaling to the "total_score" column

min_score = ads["total_score"].min()
max_score = ads["total_score"].max()

ads["normalized_score"] = (ads["total_score"] - min_score) / (max_score - min_score)

# Display the DataFrame with the normalized "total_score" column
ads["normalized_score"]
ads.to_excel("ads_score.xlsx")

In [35]:
# Sort by highest score
ads_score = ads.sort_values(by='total_score', ascending=False)
ads_score.head()

,p_date,ad_id,delivery_country,queue_market,punish_num,latest_punish_begin_date,ad_revenue,avg_ad_revenue,start_time,baseline_st,...,adrev_diff,adrev_score,st_score,p_date_dateform,days_from_latest_to_p,latest_punish_score,days_from_start_to_p,start_score,total_score,normalized_score
2258,20230807,1772501230876700,US,US&CA,0,2000-01-01,23790.20,4230.0771,2023-08-01 05:00:00,1.81,...,19560.1229,35.000000,20.496454,2023-08-07,8619,20.0,5.0,19.014891,94.511345,1.000000
1920,20230807,1770790523232290,US,US&CA,0,2000-01-01,25693.94,6904.5046,2023-08-02 12:00:00,1.81,...,18789.4354,34.167822,20.496454,2023-08-07,8619,20.0,4.0,19.037801,93.702076,0.984671
5457,20230807,1772360893228080,US,US,0,2000-01-01,18528.13,875.9772,2023-08-02 05:00:00,1.54,...,17652.1528,32.939798,21.453901,2023-08-07,8619,20.0,4.0,19.037801,93.431500,0.979545
5048,20230807,1772634217617450,FR,FR,0,2000-01-01,15306.08,395.4117,2023-07-28 04:19:00,1.76,...,14910.6683,29.979579,20.673759,2023-08-07,8619,20.0,9.0,18.923253,89.576591,0.906525
10188,20230807,1773414483579950,US,USCA,0,2000-01-01,11607.43,979.8060,2023-08-05 19:20:00,1.78,...,10627.6240,25.354802,20.602837,2023-08-07,8619,20.0,1.0,19.106529,85.064168,0.821050
